In [ ]:
import os
import json
import requests
from tqdm import tqdm
from time import sleep


# base url to reach sim
base_url = 'http://localhost:8080/'

In [ ]:
i=0
tasks = ["pyramid"]
#tasks = ["stack"]
for t in tasks:
    print("Running task: ", t)
    task_folder = f'data/llm_responses/{t}'
    for file in tqdm(sorted(os.listdir(task_folder))[:1]):
        print(f"\nRunning query {i+1}: {file}")
        # reset env
        if i != 0: requests.get(base_url + 'reset')
        
        # load data
        data = json.load(open(f"{task_folder}/{file}", 'r'))
        
        query = data['query']
        plan = data['plan']
        optimizations = data['optimizations']

        # upload plan and optimizations
        requests.post(base_url+'upload_plan', json={"query": query, "plan": plan})
        requests.post(base_url+'upload_optimizations', json={"content": optimizations})

        response = ""
        while response != "finished":
            response = requests.get(base_url+'next_task').json()[-1]['content']
            sleep(8)

        # update counter
        i+=1

_ = requests.get(base_url + 'close')

In [ ]:
task_folder = f'data/llm_responses/stack'
print(sorted(os.listdir(task_folder)))